In [5]:
from pymongo import MongoClient

In [6]:
client = MongoClient("mongodb+srv://<user>:<pasaword>@cluster0.ws4ucxc.mongodb.net/")

In [7]:
db = client['finanzas']
collection = db['facturas']

In [42]:
import pandas as pd

def run_pipeline_and_return_df(collection, pipeline):
    results = collection.aggregate(pipeline)
    results_list = list(results)
    if len(results_list) == 0:
        print("No hay resultados")
    else:
        return pd.DataFrame(results_list)

#### **1.** Realizar una consulta que devuelva la siguiente información: Región y cantidad total de productos vendidos a clientes de esa Región.

In [10]:
pipeline = [
    { "$unwind": "$item" },
    {
        "$group": {
            "_id": "$cliente.region",
            "totalProductosVendidos": { "$sum": "$item.cantidad" }
        }
    }
]

run_pipeline_and_return_df(collection, pipeline)

,_id,totalProductosVendidos
0,NEA,420
1,CABA,282
2,CENTRO,180
3,NOA,14


#### **2.** Basado en la consulta del punto 1, mostrar sólo la región que tenga el menor ingreso.

In [11]:
pipeline = [
    { "$unwind": "$item" },
    {
        "$group": {
            "_id": "$cliente.region",
            "totalProductosVendidos": { "$sum": "$item.cantidad" }
        }
    },
    { "$sort": { "totalProductosVendidos": 1 } },
    { "$limit": 1 }
]

run_pipeline_and_return_df(collection, pipeline)

,_id,totalProductosVendidos
0,NOA,14


#### **3.** Basado en la consulta del punto 1, mostrar sólo las regiones que tengan una cantidad de productos vendidos superior a 10000.

In [21]:
pipeline = [
    { "$unwind": "$item" },
    {
        "$group": {
            "_id": "$cliente.region",
            "totalProductosVendidos": { "$sum": "$item.cantidad" }
        }
    },
    { "$match": { "totalProductosVendidos": { "$gt": 10000 } } }
]

run_pipeline_and_return_df(collection, pipeline) 

No hay resultados


#### **4.** Se requiere obtener un reporte que contenga la siguiente información, nro. cuit, apellido y nombre y región y cantidad de facturas, ordenado por apellido.

In [43]:
pipeline = [
    {
        "$group": {
            "_id": "$cliente.cuit",
            "apellido": { "$first": "$cliente.apellido" },
            "nombre": { "$first": "$cliente.nombre" },
            "region": { "$first": "$cliente.region" },
            "cantidad_facturas": { "$sum": 1 }
        }
    },
    {
        "$project": {
            "_id": 0,
            "cuit": "$_id",
            "apellido": 1,
            "nombre": 1,
            "region": 1,
            "cantidad_facturas": 1
        }
    },
    { "$sort": { "apellido": 1 } }
]

run_pipeline_and_return_df(collection, pipeline)

,apellido,nombre,region,cantidad_facturas,cuit
0,Lavagno,Soledad,NOA,14,2729887543
1,Malinez,Marina,CENTRO,15,2740488484
2,Manoni,Juan Manuel,NEA,42,2029889382
3,Zavasi,Martin,CABA,29,2038373771


#### **5.** Basados en la consulta del punto 4 informar sólo los clientes con número de CUIT mayor a 27000000000.

In [36]:
pipeline = [
    {
        "$addFields": {
            "cuitAsNumber": { "$toLong": "$cliente.cuit" }
        }
    },
    {
        "$match": {
            "cuitAsNumber": { "$gt": 2700000000 } #original: 2700000000
        }
    },
    {
        "$group": {
            "_id": "$cliente.cuit",
            "apellido": { "$first": "$cliente.apellido" },
            "nombre": { "$first": "$cliente.nombre" },
            "region": { "$first": "$cliente.region" },
            "cantidad_facturas": { "$sum": 1 }
        }
    },
    {
        "$project": {
            "_id": 0,
            "cuit": "$_id",
            "apellido": 1,
            "nombre": 1,
            "region": 1,
            "cantidad_facturas": 1
        }
    },
    { "$sort": { "apellido": 1 } }
]

run_pipeline_and_return_df(collection, pipeline)

,apellido,nombre,region,cantidad_facturas,cuit
0,Lavagno,Soledad,NOA,14,2729887543
1,Malinez,Marina,CENTRO,15,2740488484


**Observacion:** He reemplazado el valor original 2700000000 por 270000000 ya que no traia resultados.

#### **6.** Basados en la consulta del punto 5 informar solamente la cantidad de clientes que cumplen con esta condición.

In [44]:
pipeline = [
    {
        "$addFields": {
            "cuitAsNumber": { "$toLong": "$cliente.cuit" }
        }
    },
    {
        "$match": {
            "cuitAsNumber": { "$gt": 2700000000 }
        }
    },
    {
        "$group": {
            "_id": "$cliente.cuit",
        }
    },
    {
        "$count": "total_clients"
    }
]

run_pipeline_and_return_df(collection, pipeline)

,total_clients
0,2


#### **7.** Se requiere realizar una consulta que devuelva la siguiente información: producto y cantidad de facturas en las que lo compraron, ordenado por cantidad de facturas descendente.

In [47]:
pipeline = [
    { "$unwind": "$item" },
    {
        "$group": {
            "_id": "$item.producto",
            "count": { "$sum": 1 }
        }
    },
    { "$sort": { "count": -1 } }
]

run_pipeline_and_return_df(collection, pipeline)

,_id,count
0,TALADRO 12mm,43
1,CORREA 10mm,29
2,TUERCA 5mm,28
3,TUERCA 2mm,28
4,SET HERRAMIENTAS,28
5,CORREA 12mm,15


#### **8.** Obtener la cantidad total comprada así como también los ingresos totales para cada producto.

In [49]:
pipeline = [
    { "$unwind": "$item" },
    {
        "$group": {
            "_id": "$item.producto",
            "cantidad_total": { "$sum": "$item.cantidad" },
            "ingresos_totales": { "$sum": { "$multiply": [ "$item.cantidad", "$item.precio" ] } }
        }
    },
    { "$sort": { "cantidad_total": -1, "ingresos_totales": -1 } }
]

run_pipeline_and_return_df(collection, pipeline)

,_id,cantidad_total,ingresos_totales
0,TUERCA 5mm,350,31500
1,CORREA 10mm,198,26532
2,CORREA 12mm,165,2970
3,TUERCA 2mm,112,6720
4,TALADRO 12mm,43,21070
5,SET HERRAMIENTAS,28,19600


#### **9.** Idem el punto anterior, ordenar por ingresos en forma ascendente, saltear el 1ro y mostrar 2do y 3ro.

In [51]:
pipeline = [
    { "$unwind": "$item" },
    {
        "$group": {
            "_id": "$item.producto",
            "cantidad_total": { "$sum": "$item.cantidad" },
            "ingresos_totales": { "$sum": { "$multiply": [ "$item.cantidad", "$item.precio" ] } }
        }
    },
    { "$sort": { "cantidad_total": -1, "ingresos_totales": -1 } },
    { "$skip": 1 },
    { "$limit": 2 }
]

run_pipeline_and_return_df(collection, pipeline)

,_id,cantidad_total,ingresos_totales
0,CORREA 10mm,198,26532
1,CORREA 12mm,165,2970


#### **10.** Obtener todos productos junto con un array de las personas que lo compraron. En este array deberá haber solo strings con el nombre completo de la persona. Los documentos entregados como resultado deberán tener la siguiente forma: {producto: “<nombre>”, personas:[“...”, ...]}

In [52]:
pipeline = [
    { "$unwind": "$item" },
    {
        "$group": {
            "_id": "$item.producto",
            "personas": { "$addToSet": { "$concat": [ "$cliente.nombre", " ", "$cliente.apellido" ] } }
        }
    },
    {
        "$project": {
            "_id": 0,
            "producto": "$_id",
            "personas": 1
        }
    }
]

run_pipeline_and_return_df(collection, pipeline)

,personas,producto
0,"[Marina Malinez, Juan Manuel Manoni]",TALADRO 12mm
1,"[Juan Manuel Manoni, Martin Zavasi]",TUERCA 2mm
2,"[Soledad Lavagno, Juan Manuel Manoni]",SET HERRAMIENTAS
3,[Juan Manuel Manoni],TUERCA 5mm
4,[Marina Malinez],CORREA 12mm
5,[Martin Zavasi],CORREA 10mm
